In [ ]:
import zarr
import numpy as np

from numcodecs import Blosc, Zstd


def random_signal_matrix(self, buffer_size):
    return np.random.uniform(-1, 1, (buffer_size,)) + 1.0j * np.random.uniform(
        -1, 1, (buffer_size,)
    )


total_samples = 2**24
buffer_size = 2**18
assert total_samples % buffer_size == 0
timesteps = total_samples // buffer_size
chunk_size = 4096
data_points = 1024
filename = "testzarr"


f64_keys = [
    "system_timestamps",
    "gps_timestamps",
    "lat",
    "long",
    "heading",
    "avg_phase_diff",
    "rssi",
    "gain",
]


def v4rx_new_dataset(filename, n_receivers, compressor=None):
    z = zarr.open(
        filename,
        mode="w",
    )
    if compressor is None:
        compressor = Blosc(
            cname="zstd",
            clevel=1,
            shuffle=Blosc.BITSHUFFLE,
        )
    z.create_group("receivers")
    for receiver_idx in range(n_receivers):
        receiver_z = z["receivers"].create_group(f"r{receiver_idx}")
        receiver_z.create_dataset(
            "signal_matrix",
            shape=(timesteps, 2, buffer_size),
            chunks=(1, 1, 1024 * chunk_size),
            dtype="complex128",
            compressor=compressor,
        )
        for key in f64_keys:
            receiver_z.create_dataset(
                key,
                shape=(timesteps,),
                chunks=(1024 * chunk_size),
                dtype="float64",
                compressor=compressor,
            )
    return z


z = v4rx_new_dataset("testdata", 2)

In [ ]:
z.tree()

In [ ]:
z.receivers.r0

In [ ]:
z.create_group("test")
z.tree()

In [ ]:
!pip install ipytree